In [147]:
import numpy as np
import pandas as pd
import cv2
import random
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from mostCommon import most_common
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import math

In [148]:
my_data = pd.read_csv('../data/Five_Frames_Per_patient.csv')
patients_data=pd.read_csv('../data/videos_data.csv')
data_array = my_data.to_numpy()
patients_array=patients_data["FileName"].to_numpy()
images_path = '../data/Five_Frames_Per_patient/'
no_of_test_patients=math.floor(patients_array.size*0.2)
images_data = []
models = []

In [149]:
rand_indexes=[]
while len(rand_indexes)<no_of_test_patients:
    h=random.randint(0, 121)
    if h not in rand_indexes:
        rand_indexes.append(h)
test_patients=np.take(patients_array,rand_indexes)
print(test)

In [150]:
#I got my patients names now i need to get their frames from my_data

df=my_data
for i in range(len(test_patients)):
    contain_values = df[df['FileName'].str.contains(test_patients[i])]['FileName'].to_numpy()
    indexes = df[df['FileName'].str.contains(test_patients[i])].index
    df.drop(indexes, inplace = True )

1830
1455
